In [1]:
import numpy as np
import pandas as pd

# A Multiply Indexed Series

## The crude way

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index = index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
# slicing
pop[('California', 2010):('Texas',2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

But you notice that the convenience ends here. You cannot, for example get only the values for the year 2010 as elegantly as we do in pandas. To do so, you would need to use a for loop:

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

This indeed produces the desired result but it is not as elegant as the Pandas-style indexing that we have grown used to. Also, it would be unoptimal for large data sets.

## The Better Way: Pandas MultiIndex

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [6]:
index.levels, index.codes

(FrozenList([['California', 'New York', 'Texas'], [2000, 2010]]),
 FrozenList([[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]]))

In [7]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [8]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

## MultiIndex as extra dimension

In [9]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [10]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Now with thsi in mind, we can use multiindexing to store three-dimensional data in a `DataFrame`:

In [11]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318933,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318933
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [12]:
f_u18 = pop_df['under18']/pop_df['total']
f_u18, type(f_u18)

(California  2000    0.273594
             2010    0.249211
 New York    2000    0.247010
             2010    0.222877
 Texas       2000    0.283251
             2010    0.273568
 dtype: float64,
 pandas.core.series.Series)

In [13]:
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222877
Texas,0.283251,0.273568


# Methods of MultiINdex Creation

The most straightforward way of creating a multiindexed `Series` or `DataFrame` is to simply pass a list of two or more index arrays to the constructor.

In [14]:
df = pd.DataFrame(np.random.rand(4,2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.940931  0.332662
  2  0.193422  0.639039
b 1  0.011405  0.720214
  2  0.893041  0.088219

In [15]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

## Explicit MultiIndex constructors

#### Using `pd.MultiIndex` constructor

In [16]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1,2,1,2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [17]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [18]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [19]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Any of these can be passed as the index in creating a `Series` or `DataFrame`, or be passed to the `reindex` method of an existing `Series` or `DataFrame`.

## MultiIndex code names

In [20]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## MultiIndex for columns

In `DataFrame`, the rows and columns are completely symmetric, and just as rows can have multiple levels of indices, so also columns can have multiple levels of indeices. Consider the following mock-up of some medical data:

In [21]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names = ['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4,6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      31.0  37.4  37.0  35.9  25.0  37.7
     2      39.0  37.4  40.0  36.8  40.0  36.5
2014 1      30.0  35.9  47.0  36.2  46.0  37.1
     2      49.0  36.2  25.0  35.0  20.0  37.3

In escence, this is a four-dimensional data, where the dimensions are *year, visit, subject, type*. Multiindexing comes very in handy esp in such a case. For example we can get a full `DataFrame` containing just that person's information:

In [22]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      37.0  35.9
     2      40.0  36.8
2014 1      47.0  36.2
     2      25.0  35.0

# Indexing and Slicing a MultiIndex

## Multiply indexed Series

In [23]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [24]:
pop['California', 2000]

33871648

In [25]:
# partial indexing
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [26]:
# you can also slice for sorted MultiIndex
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [27]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [28]:
pop[pop>22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [29]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

## Multiply indexed DataFrames

In [30]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      31.0  37.4  37.0  35.9  25.0  37.7
     2      39.0  37.4  40.0  36.8  40.0  36.5
2014 1      30.0  35.9  47.0  36.2  46.0  37.1
     2      49.0  36.2  25.0  35.0  20.0  37.3

In [31]:
health_data['Guido', 'HR']

year  visit
2013  1        37.0
      2        40.0
2014  1        47.0
      2        25.0
Name: (Guido, HR), dtype: float64

In [32]:
health_data['Guido', 'HR'][2013]

visit
1    37.0
2    40.0
Name: (Guido, HR), dtype: float64

In [33]:
health_data['Guido', 'HR'][:,1]

year
2013    37.0
2014    47.0
Name: (Guido, HR), dtype: float64

In [34]:
health_data['Guido', 'HR'][2014,2]

25.0

In [35]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      31.0  37.4
     2      39.0  37.4

In [36]:
health_data.loc[2013,('Guido', 'HR'),].loc[1]

37.0

In [37]:
idx = pd.IndexSlice
health_data.loc[idx[:],  idx[:,"HR"]]

subject      Bob Guido   Sue
type          HR    HR    HR
year visit                  
2013 1      31.0  37.0  25.0
     2      39.0  40.0  40.0
2014 1      30.0  47.0  46.0
     2      49.0  25.0  20.0

In [38]:
# all temperature records
health_data.loc[idx[:], idx[:,'Temp']]

subject      Bob Guido   Sue
type        Temp  Temp  Temp
year visit                  
2013 1      37.4  35.9  37.7
     2      37.4  36.8  36.5
2014 1      35.9  36.2  37.1
     2      36.2  35.0  37.3

In [39]:
# all 2014 HR records
health_data.loc[idx[2014], idx[:,'HR']]

subject,Bob,Guido,Sue
type,HR,HR,HR
visit,,,
1,30.0,47.0,46.0
2,49.0,25.0,20.0


In [40]:
# all visit 1 Temp records
health_data.loc[idx[:,1], idx[:,'Temp']]

,subject,Bob,Guido,Sue
,type,Temp,Temp,Temp
year,visit,,,
2013,1,37.4,35.9,37.7
2014,1,35.9,36.2,37.1


# Rearranging Multi-Indices

## Sorted and unsorted indices

**NOTE:** *Many of the `MultiIndex` slicing operators will fail if teh index is not sorted*

In [41]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.569774
      2      0.760231
c     1      0.341104
      2      0.685640
b     1      0.178760
      2      0.582150
dtype: float64

In [42]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


This results from the fact that the MultiIndex is not sorted. Thus, the indices need to be sorted for the slices to operated on them.

In [43]:
data = data.sort_index()
data

char  int
a     1      0.569774
      2      0.760231
b     1      0.178760
      2      0.582150
c     1      0.341104
      2      0.685640
dtype: float64

In [44]:
data['a':'b']

char  int
a     1      0.569774
      2      0.760231
b     1      0.178760
      2      0.582150
dtype: float64

## Stacking and unstacking indices

In [45]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [46]:
pop.unstack(level = 0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [47]:
pop.unstack(level = 1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [48]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## Index setting and resetting

# index resetting
pop_flat = pop.reset_index(name='population')
pop_flat

In [50]:
# index setting
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

This is a very important aspect when it comes to real-world datasets. It easily creates indices.

# Data Aggregation on Multi-Indices

In [51]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      31.0  37.4  37.0  35.9  25.0  37.7
     2      39.0  37.4  40.0  36.8  40.0  36.5
2014 1      30.0  35.9  47.0  36.2  46.0  37.1
     2      49.0  36.2  25.0  35.0  20.0  37.3

Let us say we want to average the measurements in the two visits each year. THis can be done by naming theindex level we'd like to explore, in this case, the year:

In [52]:
data_mean = health_data.groupby(level='year').mean()
data_mean

subject   Bob        Guido          Sue      
type       HR   Temp    HR   Temp    HR  Temp
year                                         
2013     35.0  37.40  38.5  36.35  32.5  37.1
2014     39.5  36.05  36.0  35.60  33.0  37.2

In [53]:
data_mean.groupby(axis=1, level='type').mean()

type,HR,Temp
year,,
2013,35.333333,36.950000
2014,36.166667,36.283333


In [54]:
data_mean.groupby(axis=0, level='year').mean()

subject   Bob        Guido          Sue      
type       HR   Temp    HR   Temp    HR  Temp
year                                         
2013     35.0  37.40  38.5  36.35  32.5  37.1
2014     39.5  36.05  36.0  35.60  33.0  37.2